1. Define tools and model
In this example, we’ll use the OpenAI GPT-4.1-mini model and define tools for addition, multiplication, and division.

In [1]:
# Just imports
import os
import getpass
from langchain.tools import tool
from langchain.chat_models import init_chat_model
from langgraph.graph import add_messages
from langchain.messages import (
    SystemMessage,
    HumanMessage,
    ToolCall,
)
from langchain_core.messages import BaseMessage
from langgraph.func import entrypoint, task

In [2]:
# Set OpenAI API key
if not os.environ.get("OPENAI_API_KEY"):
    # Prompt user for API key securely
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
# Initialize the chat model with specified parameters
model = init_chat_model("gpt-4.1-mini", temperature=0.5, max_retries=3, timeout=60, max_tokens=1000, streaming=True)

In [3]:
# Define tools
@tool
def multiply(a: int, b: int) -> int:
    """Multiply `a` and `b`.

    Args:
        a: First int
        b: Second int
    """
    return a * b


@tool
def add(a: int, b: int) -> int:
    """Adds `a` and `b`.

    Args:
        a: First int
        b: Second int
    """
    return a + b


@tool
def divide(a: int, b: int) -> float:
    """Divide `a` and `b`.

    Args:
        a: First int
        b: Second int
    """
    return a / b

In [4]:
# Augment the LLM with tools
tools = [add, multiply, divide]
# Create a mapping of tool names to tool instances
tools_by_name = {tool.name: tool for tool in tools}
model_with_tools = model.bind_tools(tools)

2. Define model node
The model node is used to call the LLM and decide whether to call a tool or not.
The @task decorator marks a function as a task that can be executed as part of the agent. Tasks can be called synchronously or asynchronously within your entrypoint function.

In [5]:
@task
def call_llm(messages: list[BaseMessage]):
    """LLM decides whether to call a tool or not."""
    # Prepare the messages for the LLM invocation
    return model_with_tools.invoke(
        [
            SystemMessage(
                content="You are a helpful assistant tasked with performing arithmetic on a set of inputs."
            )
        ]
        + messages
    )

3. Define tool node
The tool node is used to call the tools and return the results.

In [6]:
@task
def call_tool(tool_call: ToolCall):
    """Performs the tool call"""
    tool = tools_by_name[tool_call["name"]]
    return tool.invoke(tool_call)

4. Define agent
The agent is built using the @entrypoint function.
In the Functional API, instead of defining nodes and edges explicitly, you write standard control flow logic (loops, conditionals) within a single function.

In [8]:
@entrypoint()
def agent(messages: list[BaseMessage]):
    # Initial LLM call
    model_response = call_llm(messages).result()
    # Loop until there are no more tool calls
    while True:

        if not model_response.tool_calls:
            break

        # Execute tools
        tool_result_futures = [
            call_tool(tool_call) for tool_call in model_response.tool_calls
        ]
        # Gather tool results
        tool_results = [fut.result() for fut in tool_result_futures]
        # Add model response and tool results to messages
        messages = add_messages(messages, [model_response, *tool_results])
        # Update the model response with new tool results
        model_response = call_llm(messages).result()

    # Final addition of the model response
    messages = add_messages(messages, model_response)
    return messages

In [9]:
# Invoke the agent with a sample input
messages = [HumanMessage(content="Add 3 and 4.")]
for chunk in agent.stream(messages, stream_mode="updates"):
    print(chunk)
    print("\n")

{'call_llm': AIMessage(content='', additional_kwargs={}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_9766e549b2', 'service_tier': 'default', 'model_provider': 'openai'}, id='lc_run--f0ff335f-76f5-4c52-9174-706e167565be', tool_calls=[{'name': 'add', 'args': {'a': 3, 'b': 4}, 'id': 'call_XuM4XuZlN7xAC7Kxm6PBsJZ7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 162, 'output_tokens': 17, 'total_tokens': 179, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})}


{'call_tool': ToolMessage(content='7', name='add', id='4987895e-fe42-4d58-ab75-f449c1e42746', tool_call_id='call_XuM4XuZlN7xAC7Kxm6PBsJZ7')}


{'call_llm': AIMessage(content='3 plus 4 is 7.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_9766e549b2', 'service_tier': 'default', 'model_provider': 'openai'}, id='l